<a href="https://colab.research.google.com/github/agam844/Active-Learning/blob/main/keras_CIPHAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
import random

Training on the entire dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [ ]:
# Preprocess the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer= optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')

model.fit(x_train, y_train, batch_size=64, epochs=100, validation_data=(x_test, y_test), callbacks=[early_stopping])

Epoch 1/100
782/782 [==============================] - 16s 6ms/step - loss: 1.7800 - accuracy: 0.3494 - val_loss: 1.4234 - val_accuracy: 0.5012
Epoch 2/100
782/782 [==============================] - 4s 5ms/step - loss: 1.4744 - accuracy: 0.4666 - val_loss: 1.2832 - val_accuracy: 0.5410
Epoch 3/100
782/782 [==============================] - 5s 6ms/step - loss: 1.3687 - accuracy: 0.5072 - val_loss: 1.1980 - val_accuracy: 0.5725
Epoch 4/100
782/782 [==============================] - 4s 5ms/step - loss: 1.3064 - accuracy: 0.5341 - val_loss: 1.1307 - val_accuracy: 0.6064
Epoch 5/100
782/782 [==============================] - 4s 5ms/step - loss: 1.2527 - accuracy: 0.5540 - val_loss: 1.1043 - val_accuracy: 0.6109
Epoch 6/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2102 - accuracy: 0.5686 - val_loss: 1.0484 - val_accuracy: 0.6320
Epoch 7/100
782/782 [==============================] - 4s 6ms/step - loss: 1.1849 - accuracy: 0.5807 - val_loss: 1.0625 - val_accuracy: 0.635

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

313/313 [==============================] - 1s 3ms/step - loss: 0.9143 - accuracy: 0.6783
Test loss: 0.9143
Test accuracy: 0.6783


In [ ]:
# Get predicted probabilities for test data
probabilities = model.predict(x_test)

# Get the class labels with the highest probability
predictions_int = np.argmax(probabilities, axis=1)

# Get predicted probabilities for test data
probabilities = model.predict(x_test)

# Get the highest probability for each sample
max_probabilities = np.max(probabilities, axis=1)

313/313 [==============================] - 1s 2ms/step


In [ ]:
print(np.sort(max_probabilities)[:10])
len(max_probabilities)

[0.14337356 0.14367919 0.14433481 0.14860065 0.15034115 0.15617497
 0.15620431 0.15938121 0.15965697 0.16260476]


10000

Training using active Learning

In [ ]:
num_entries = min(5000, len(y_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

In [ ]:
# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

In [ ]:
remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

In [ ]:
(x_train1, y_train1), (x_test1, y_test1) = cifar10.load_data()
remaining_x_train1 = x_train1[remaining_indices]
remaining_y_train1 = y_train1[remaining_indices]

In [ ]:
#active learning loop
num_iterations = 10
for iteration in range(num_iterations):
  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

#   model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  model.fit(selected_x_train, selected_y_train, batch_size=64, epochs=10)
  loss, accuracy = model.evaluate(remaining_x_train, remaining_y_train)
  print(f'Test loss: {loss:.4f}')
  print(f'Test accuracy: {accuracy:.4f}')
  #Select additional samples from remaining entries for labeling
  num_additional_samples = 2000

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the class labels with the highest probability
  predictions_int = np.argmax(probabilities, axis=1)

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the highest probability for each sample
  max_probabilities = np.max(probabilities, axis=1)

  for i in range(len(predictions_int)):
    if predictions_int[i] != remaining_y_train1[i]:
      max_probabilities[i] = 0

  #additional_indices = np.argsort(np.max(predictions, axis=1))[-num_additional_samples:]
  num_additional_samples = 2000
  # additional_indices = np.argsort(np.max(max_probabilities))[:num_additional_samples]
  additional_indices = np.argsort(max_probabilities)[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)
  remaining_y_train1 = np.delete(remaining_y_train1, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

Epoch 1/10
79/79 [==============================] - 2s 7ms/step - loss: 2.2395 - accuracy: 0.1508
Epoch 2/10
79/79 [==============================] - 0s 5ms/step - loss: 2.0261 - accuracy: 0.2464
Epoch 3/10
79/79 [==============================] - 0s 5ms/step - loss: 1.8620 - accuracy: 0.3120
Epoch 4/10
79/79 [==============================] - 0s 4ms/step - loss: 1.7917 - accuracy: 0.3434
Epoch 5/10
79/79 [==============================] - 0s 5ms/step - loss: 1.7311 - accuracy: 0.3650
Epoch 6/10
79/79 [==============================] - 0s 5ms/step - loss: 1.6778 - accuracy: 0.3842
Epoch 7/10
79/79 [==============================] - 0s 5ms/step - loss: 1.6327 - accuracy: 0.3968
Epoch 8/10
79/79 [==============================] - 0s 5ms/step - loss: 1.5830 - accuracy: 0.4218
Epoch 9/10
79/79 [==============================] - 0s 6ms/step - loss: 1.5661 - accuracy: 0.4346
Epoch 10/10
1407/1407 [==============================] - 5s 3ms/step - loss: 1.4999 - accuracy: 0.4709
Test loss: 1.49

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(selected_x_train, selected_y_train, batch_size=64, epochs=10)

Epoch 1/10
391/391 [==============================] - 5s 6ms/step - loss: 2.1050 - accuracy: 0.1971
Epoch 2/10
391/391 [==============================] - 2s 5ms/step - loss: 1.8888 - accuracy: 0.2631
Epoch 3/10
391/391 [==============================] - 2s 5ms/step - loss: 1.8137 - accuracy: 0.2886
Epoch 4/10
391/391 [==============================] - 2s 5ms/step - loss: 1.7573 - accuracy: 0.3083
Epoch 5/10
391/391 [==============================] - 2s 5ms/step - loss: 1.7092 - accuracy: 0.3280
Epoch 6/10
391/391 [==============================] - 2s 6ms/step - loss: 1.6738 - accuracy: 0.3456
Epoch 7/10
391/391 [==============================] - 2s 6ms/step - loss: 1.6397 - accuracy: 0.3586
Epoch 8/10
391/391 [==============================] - 2s 5ms/step - loss: 1.6083 - accuracy: 0.3730
Epoch 9/10
391/391 [==============================] - 2s 5ms/step - loss: 1.5794 - accuracy: 0.3867
Epoch 10/10
391/391 [==============================] - 2s 5ms/step - loss: 1.5547 - accuracy: 0.3979

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

313/313 [==============================] - 1s 3ms/step - loss: 1.2045 - accuracy: 0.5968
Test loss: 1.2045
Test accuracy: 0.5968


Without correctness

In [ ]:
num_entries = min(5000, len(y_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

(x_train1, y_train1), (x_test1, y_test1) = cifar10.load_data()
remaining_x_train1 = x_train1[remaining_indices]
remaining_y_train1 = y_train1[remaining_indices]

In [ ]:
#active learning loop
num_iterations = 10
for iteration in range(num_iterations):
  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

#   model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  model.fit(selected_x_train, selected_y_train, batch_size=64, epochs=10)
  loss, accuracy = model.evaluate(remaining_x_train, remaining_y_train)
  print(f'Test loss: {loss:.4f}')
  print(f'Test accuracy: {accuracy:.4f}')

  #additional_indices = np.argsort(np.max(predictions, axis=1))[-num_additional_samples:]
  num_additional_samples = 2000
  # additional_indices = np.argsort(np.max(max_probabilities))[:num_additional_samples]
  additional_indices = np.argsort(max_probabilities)[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)
  remaining_y_train1 = np.delete(remaining_y_train1, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(selected_x_train, selected_y_train, batch_size=64, epochs=10)

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

Different Baseline

In [ ]:
num_entries = min(10000, len(y_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

(x_train1, y_train1), (x_test1, y_test1) = cifar10.load_data()
remaining_x_train1 = x_train1[remaining_indices]
remaining_y_train1 = y_train1[remaining_indices]

In [ ]:
#active learning loop
num_iterations = 10
for iteration in range(num_iterations):
  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

#   model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  model.fit(selected_x_train, selected_y_train, batch_size=64, epochs=10)
  loss, accuracy = model.evaluate(remaining_x_train, remaining_y_train)
  print(f'Test loss: {loss:.4f}')
  print(f'Test accuracy: {accuracy:.4f}')

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the class labels with the highest probability
  predictions_int = np.argmax(probabilities, axis=1)

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the highest probability for each sample
  max_probabilities = np.max(probabilities, axis=1)

  for i in range(len(predictions_int)):
    if predictions_int[i] != remaining_y_train1[i]:
      max_probabilities[i] = 0

  #additional_indices = np.argsort(np.max(predictions, axis=1))[-num_additional_samples:]
  num_additional_samples = 1000
  # additional_indices = np.argsort(np.max(max_probabilities))[:num_additional_samples]
  additional_indices = np.argsort(max_probabilities)[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)
  remaining_y_train1 = np.delete(remaining_y_train1, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

Epoch 1/10
157/157 [==============================] - 2s 5ms/step - loss: 2.1489 - accuracy: 0.1975
Epoch 2/10
157/157 [==============================] - 1s 5ms/step - loss: 1.8099 - accuracy: 0.3341
Epoch 3/10
157/157 [==============================] - 1s 5ms/step - loss: 1.6548 - accuracy: 0.3943
Epoch 4/10
157/157 [==============================] - 1s 5ms/step - loss: 1.5706 - accuracy: 0.4302
Epoch 5/10
157/157 [==============================] - 1s 5ms/step - loss: 1.5119 - accuracy: 0.4582
Epoch 6/10
157/157 [==============================] - 1s 5ms/step - loss: 1.4447 - accuracy: 0.4748
Epoch 7/10
157/157 [==============================] - 1s 5ms/step - loss: 1.4081 - accuracy: 0.4911
Epoch 8/10
157/157 [==============================] - 1s 6ms/step - loss: 1.3720 - accuracy: 0.5086
Epoch 9/10
157/157 [==============================] - 1s 6ms/step - loss: 1.3464 - accuracy: 0.5188
Epoch 10/10
1250/1250 [==============================] - 4s 3ms/step - loss: 1.2628 - accuracy: 0.55

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(selected_x_train, selected_y_train, batch_size=64, epochs=10)

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')